# Avaliação - Bancos de Dados I

**Aluno:** Gabriel Resende Miranda

**Professor:** Lucas Ximenes

## Modelagem e normalização de bancos de dados relacionais

Certo dia, um dos gestores do banco em que você trabalha como cientista de dados procurou você pedindo ajuda para projetar um pequeno banco de dados com o objetivo de mapear os clientes da companhia pelos diferentes produtos financeiros que eles contrataram.

O gestor explicou que o banco tinha uma grande quantidade de clientes e oferecia uma variedade de produtos financeiros, como cartões de crédito, empréstimos, seguros e investimentos. No entanto, eles estavam tendo dificuldades para entender quais produtos eram mais populares entre os clientes e como esses produtos estavam interagindo entre si.

Como ponto de partida, o gestor deixou claro que um cliente pode contratar vários produtos diferentes ao passo que um mesmo produto pode também estar associado a vários clientes diferentes e elaborou um rústico esboço de banco de dados com duas tabelas, da seguinte forma:

* **Tabela 1**
    * **Nome da tabela:** cliente
    * **Colunas:** codigo_cliente, nome_cliente, sobrenome_cliente, telefones_cliente, municipio_cliente, codigo_tipo_cliente, tipo_cliente
* **Tabela 2**
    * **Nome da tabela:** produto
    * **Colunas:** codigo_produto, nome_produto, descricao_produto, codigo_tipo_produto, tipo_produto, codigo_diretor_responsavel, nome_diretor_responsavel, email_diretor_responsavel

### Exercício 1

_1) Ainda sem fazer normalizações, apresente o modelo conceitual deste esboço oferecido pelo gestor, destacando atributos chaves e multivalorados, caso existam, e apresentando também a cardinalidade dos relacionamentos._

![Exercício 1](img/exercicio1.png)

Nesse caso, **o único atributo multivalorado** é o atributo ```telefones_cliente``` da tabela ```cliente```.

### Exercício 2

_2) Agora apresente um modelo lógico que expresse as mesmas informações e relacionamentos descritos no modelo original, mas decompondo-os quando necessário para que sejam respeitadas as 3 primeiras formas normais. Destaque atributos chaves e multivalorados, caso existam, e apresente também a cardinalidade dos relacionamentos._

Para facilitar a visualização, irei antes construir um **modelo conceitual** respeitando as regras de normalização.

![Exercício 2 (conceitual)](img/exercicio2_m_conceitual.png)

A partir do modelo conceitual, construí o **modelo lógico** utilizando o software BrModeloWeb:

![Exercício 2 (lógico)](img/exercicio2_m_logico.png)

A implementação de um banco de dados seguindo essa estrutura pode ser feita com o seguinte comando SQL (`sql/exercicio2.sql`):

```sql
CREATE TABLE tipo_cliente 
( 
    codigo_tipo_cliente INT PRIMARY KEY AUTO_INCREMENT,  
    descricao_tipo_cliente VARCHAR(200) NOT NULL
); 

CREATE TABLE cliente 
( 
    codigo_cliente INT PRIMARY KEY AUTO_INCREMENT,  
    nome_cliente VARCHAR(200) NOT NULL,  
    sobrenome_cliente VARCHAR(200) NOT NULL,  
    municipio_cliente VARCHAR(200) NOT NULL,  
    codigo_tipo_cliente INT,
    FOREIGN KEY(codigo_tipo_cliente) REFERENCES tipo_cliente(codigo_tipo_cliente)
); 

CREATE TABLE telefone 
( 
    codigo_telefone INT PRIMARY KEY AUTO_INCREMENT,  
    numero_telefone VARCHAR(20) NOT NULL,
    codigo_cliente INT,
    FOREIGN KEY(codigo_cliente) REFERENCES cliente(codigo_cliente)
); 

CREATE TABLE tipo_produto 
( 
    codigo_tipo_produto INT PRIMARY KEY AUTO_INCREMENT,  
    descricao_tipo_produto VARCHAR(200) NOT NULL
); 

CREATE TABLE diretor_responsavel 
( 
    codigo_diretor_responsavel INT PRIMARY KEY AUTO_INCREMENT,  
    nome_diretor_responsavel VARCHAR(200) NOT NULL,  
    email_diretor_responsavel VARCHAR(200) NOT NULL
); 

CREATE TABLE produto 
( 
    codigo_produto INT PRIMARY KEY AUTO_INCREMENT,  
    nome_produto VARCHAR(200) NOT NULL,  
    descricao_produto VARCHAR(200) NOT NULL,
    codigo_tipo_produto INT,
    codigo_diretor_responsavel INT,
    FOREIGN KEY(codigo_tipo_produto) REFERENCES tipo_produto(codigo_tipo_produto),
    FOREIGN KEY(codigo_diretor_responsavel) REFERENCES diretor_responsavel(codigo_diretor_responsavel)
); 

CREATE TABLE contrato 
( 
    codigo_cliente INT,  
    codigo_produto INT,  
    PRIMARY KEY(codigo_cliente, codigo_produto),
    FOREIGN KEY(codigo_cliente) REFERENCES cliente(codigo_cliente),
    FOREIGN KEY(codigo_produto) REFERENCES produto(codigo_produto)
);
```

## Consultas SQL simples e complexas em um banco de dados MySQL

Um exemplo de modelo de banco de dados com relacionamento muitos-para-muitos pode ser o de um e-commerce que tem produtos e categorias, onde um produto pode pertencer a várias categorias e uma categoria pode estar associada a vários produtos. Nesse caso, teríamos duas tabelas: "produtos" e "categorias", com uma tabela intermediária "produtos_categorias" para relacionar os produtos às suas categorias.

```sql
CREATE TABLE produto (
    id_produto INT AUTO_INCREMENT PRIMARY KEY,
    nome_produto VARCHAR(100) NOT NULL,
    preco_produto DECIMAL(10, 2) NOT NULL
);

CREATE TABLE categoria (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nome_categoria VARCHAR(100) NOT NULL
);

CREATE TABLE produto_categoria (
    id_produto INT,
    id_categoria INT,
    PRIMARY KEY (id_produto, id_categoria),
    FOREIGN KEY(id_produto) REFERENCES produto(id_produto),
    FOREIGN KEY(id_categoria) REFERENCES categoria(id_categoria)
);
```

Assim, usando o subconjunto da "structured query language" chamado de DQL, produza consultas postgress de modo a atender cada uma das seguintes solicitações:

### Exercício 3

_3) Liste os nomes de todos os produtos que custam mais de 100 reais, ordenando-os primeiramente pelo preço e em segundo lugar pelo nome. Use alias para mostrar o nome da coluna nome como "Produto" e da coluna preco como "Valor". A resposta da consulta não deve mostrar outras colunas de dados._

Primeiramente, executei o código do enunciado para criar as tabelas (pode ser encontrado em `sql/exercicio3_criando_tabelas`).

Após, populei a tabela com algumas informações fictícias para testar o funcionamento (o código pode ser encontrado em `sql/exercicio3_preenchendo_tabelas`).

A query para esse exercício fica (`sql/exercicio3_query`):

```sql
SELECT
    nome_produto AS "Produto",
    preco_produto AS "Valor"
FROM produto
WHERE preco_produto > 100
ORDER BY preco_produto, nome_produto;
```

### Exercício 4

_4) Liste todos os ids e preços de produtos cujo preço seja maior do que a média de todos os preços encontrados na tabela "produtos"._

A query para esse exercício fica (`sql/exercicio4_query`):

```sql
SELECT
    id_produto,
    preco_produto 
FROM produto 
WHERE preco_produto > (SELECT AVG(preco_produto) FROM produto);
```

### Exercício 5

_5) Para cada categoria, mostre o preço médio do conjunto de produtos a ela associados. Caso uma categoria não tenha nenhum produto a ela associada, esta categoria não deve aparecer no resultado final. A consulta deve estar ordenada pelos nomes das categorias._

A query para esse exercício fica (`sql/exercicio5_query`):

```sql
SELECT 
    c.nome_categoria, 
    AVG(p.preco_produto) AS preco_medio
FROM 
    categoria c
INNER JOIN 
    produto_categoria pc ON c.id_categoria = pc.id_categoria
INNER JOIN 
    produto p ON pc.id_produto = p.id_produto
GROUP BY 
    c.nome_categoria
ORDER BY 
    c.nome_categoria;
```

## Inserções, alterações e remoções de objetos e dados em um banco de dados MySQL

Você está participando de um processo seletivo para trabalhar como cientista de dados na Ada, uma das maiores formadoras do país em áreas correlatadas à tecnologia. Dividido em algumas etapas, o processo tem o objetivo de avaliar você nos quesitos Python, Machine Learning e Bancos de Dados. Ainda que os dois primeiros sejam o cerne da sua atuação no dia-a-dia, considera-se que Bancos de Dados também constituem um requisito importante e, por isso, esta etapa pode ser a oportunidade que você precisava para se destacar dentre os seus concorrentes, demonstrando um conhecimento mais amplo do que os demais.

### Exercício 6

_6) Com o objetivo de demonstrar o seu conhecimento através de um exemplo contextualizado com o dia-a-dia da escola, utilize os comandos do subgrupo de funções DDL para construir o banco de dados simples abaixo, que representa um relacionamento do tipo 1,n entre as entidades "aluno" e "turma":_

* **Tabela 1:**
    * **Nome da tabela:** aluno
    * **Colunas da tabela:** id_aluno (INT), nome_aluno (VARCHAR), aluno_alocado (BOOLEAN), id_turma (INT)
* **Tabela 2:**
    * **Nome da tabela:** turma
    * **Colunas da tabela:** id_turma (INT), código_turma (VARCHAR), nome_turma (VARCHAR)

Primeiramente, construí os modelos conceitual e lógico referentes a esse exercício:

![Exercício 6 (modelo conceitual)](img/exercicio6_m_conceitual.png)

![Exercício 6 (modelo lógico)](img/exercicio6_m_logico.png)

Os comandos para criação dessa tabela são (`sql/exercicio6_criando_tabelas`):

```sql
CREATE TABLE turma (
    id_turma INT AUTO_INCREMENT PRIMARY KEY,
    codigo_turma VARCHAR(100) NOT NULL,
    nome_turma VARCHAR(100) NOT NULL
);

CREATE TABLE aluno (
    id_aluno INT AUTO_INCREMENT PRIMARY KEY,
    nome_aluno VARCHAR(255) NOT NULL,
    aluno_alocado BOOLEAN NOT NULL DEFAULT FALSE,
    id_turma INT,
    FOREIGN KEY(id_turma) REFERENCES turma(id_turma)
);
```

Note que, por padrão, coloquei os valores de `aluno.aluno_alocado = FALSE`.

### Exercício 7

_7) Agora que você demonstrou que consegue ser mais do que um simples usuário do banco de dados, mostre separadamente cada um dos códigos DML necessários para cumprir cada uma das etapas a seguir:_

_a) Inserir pelo menos duas turmas diferentes na tabela de turma;_
```sql
INSERT INTO turma (codigo_turma, nome_turma) VALUES
("BD001", "Banco de Dados I"),
("ES001", "Estatística"),
("DS001", "Lógica de Programação"),
("DS002", "Técnicas de Programação"),
("DS003", "Machine Learning");
```

_b) Inserir pelo menos 1 aluno alocado em cada uma destas turmas na tabela aluno (todos com FALSE na coluna aluno_alocado);_
```sql
INSERT INTO aluno (nome_aluno, id_turma) VALUES
("Pedro", 1),
("Mariana", 2),
("Lucas", 3),
("Aline", 4),
("Henrique", 5);
```

_c) Inserir pelo menos 2 alunos não alocados em nenhuma turma na tabela aluno (todos com FALSE na coluna aluno_alocado);_
```sql
INSERT INTO aluno (nome_aluno) VALUES
("Gabriela"),
("Rafael");
```

_d) Atualizar a coluna aluno_alocado da tabela aluno, de modo que os alunos associados a uma disciplina recebam o valor True e alunos não associdos a nenhuma disciplina recebam o falor False para esta coluna._
```sql
UPDATE aluno 
SET aluno_alocado = TRUE 
WHERE id_turma IS NOT NULL;

UPDATE aluno 
SET aluno_alocado = FALSE 
WHERE id_turma IS NULL;
```

Todos os comandos acima estão no arquivo `sql/exercicio7`.